<a href="https://colab.research.google.com/github/morioka/BERT_Japanese_Google_Colaboratory/blob/master/1_Japanese_BERT_on_Google_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Qiita記事：【実装解説】日本語版BERTをGoogle Colaboratoryで使う方法の実装コード

URL：https://qiita.com/sugulu_Ogawa_ISID/items/e522a38b812b8edb8a54


## 準備1：MeCabをGoogle Colaboratoryにインストール

In [1]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y


Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.8)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.13)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already instal

In [2]:
!pip install mecab-python3
!pip install fugashi ipadic

In [3]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :     tar => ok
[install-mecab-ipadic-NEologd] :     unxz => ok
[install-mecab-ipadic-NEologd] :     xargs => ok
[install-mecab-ipadic-NEologd] :     grep => ok
[install-mecab-ipadic-NEo

In [4]:
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

In [5]:
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

ln: failed to create symbolic link '/usr/local/etc/mecabrc': File exists


## 準備2：MeCabの動作確認

In [6]:
import MeCab

m=MeCab.Tagger("-Ochasen")

text = "私は機械学習が好きです。"

text_segmented = m.parse(text)
print(text_segmented)

私	ワタシ	私	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [7]:
m=MeCab.Tagger("-Owakati")
text_segmented = m.parse(text)
print(text_segmented)

私 は 機械 学習 が 好き です 。 



In [8]:
m=MeCab.Tagger("-Oyomi")
text_segmented = m.parse(text)
print(text_segmented)

ワタシハキカイガクシュウガスキデス。



### 新語辞書の場合

In [9]:
m=MeCab.Tagger("-Ochasen -d "+str(path_neologd))  # NEologdへのパスを追加

text = "私は機械学習が好きです。"

text_segmented = m.parse(text)
print(text_segmented)

私	ワタシ	私	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
機械学習	キカイガクシュウ	機械学習	名詞-固有名詞-一般		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [10]:
m=MeCab.Tagger("-Owakati -d "+str(path_neologd))  # NEologdへのパスを追加
text_segmented = m.parse(text)
print(text_segmented)

私 は 機械学習 が 好き です 。 



## 準備3：日本語版BERTの学習済みモデルと形態素解析を用意

In [11]:
!pip install transformers==4.5.1

In [12]:
import torch
from transformers import BertModel
from transformers import BertJapaneseTokenizer


In [13]:
# 分かち書きをするtokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


In [14]:
# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
print(model)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [15]:
from transformers import BertConfig

# 東北大学_日本語版の設定を確認
config_japanese = BertConfig.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
print(config_japanese)


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.5.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



設定を見ると、単語ベクトルは768次元、最大の単語数（サブワード数）は512、BERTのレイヤー数は12層、ボキャブラリのサイズは32,000であることが分かります。

## 日本語版BERTで文章を扱う

In [16]:
text1 = "会社をクビになった。"
text2 = "テレワークばかりでクビが痛い。"
text3 = "会社を解雇された。"


In [17]:
# 分かち書きをして、idに変換
input_ids1 = tokenizer.encode(text1, return_tensors='pt')  # ptはPyTorchの略

print(tokenizer.convert_ids_to_tokens(input_ids1[0].tolist()))  # 文章
print(input_ids1)  # id


['[CLS]', '会社', 'を', 'クビ', 'に', 'なっ', 'た', '。', '[SEP]']
tensor([[    2,   811,    11, 13700,     7,    58,    10,     8,     3]])


In [18]:
# 分かち書きをして、idに変換
input_ids2 = tokenizer.encode(text2, return_tensors='pt')  # ptはPyTorchの略

print(tokenizer.convert_ids_to_tokens(input_ids2[0].tolist()))  # 文章
print(input_ids2)  # id


['[CLS]', 'テレ', '##ワーク', 'ばかり', 'で', 'クビ', 'が', '痛', '##い', '。', '[SEP]']
tensor([[    2,  5521,  3118,  4027,    12, 13700,    14,  4897, 28457,     8,
             3]])


In [19]:
# 分かち書きをして、idに変換
input_ids3 = tokenizer.encode(text3, return_tensors='pt')  # ptはPyTorchの略

print(tokenizer.convert_ids_to_tokens(input_ids3[0].tolist()))  # 文章
print(input_ids3)  # id

['[CLS]', '会社', 'を', '解雇', 'さ', 'れ', 'た', '。', '[SEP]']
tensor([[   2,  811,   11, 7279,   26,   20,   10,    8,    3]])


In [20]:
# 日本語BERTモデルに入力
result1 = model(input_ids1)

print(result1[0].shape)
print(result1[1].shape)

# reult は、sequence_output, pooled_output, (hidden_states), (attentions)です。
# ただし、hidden_statesとattentionsはoptionalであり、標準では出力されません。


torch.Size([1, 9, 768])
torch.Size([1, 768])


In [21]:
# 日本語BERTモデルに入力
result2 = model(input_ids2)
result3 = model(input_ids3)

word_vec1 = result1[0][0][3][:]  # 1つ目の文章の”クビ”（3番目）
word_vec2 = result2[0][0][5][:]  # 2つ目の文章の”クビ”（5番目）
word_vec3 = result3[0][0][3][:]  # 3つ目の文章の”解雇”（3番目）

In [22]:
# コサイン類似度を求める
cos = torch.nn.CosineSimilarity(dim=0)
cos_sim_12 = cos(word_vec1, word_vec2)
cos_sim_13 = cos(word_vec1, word_vec3)

print(cos_sim_12)
print(cos_sim_13)

tensor(0.6647, grad_fn=<DivBackward0>)
tensor(0.7841, grad_fn=<DivBackward0>)
